In [ ]:
from __future__ import annotations

import re

from functools import reduce

In [ ]:
line = "[#.#.#.#.#.] (0,1,2,3,5,6,8,9) (1,4,6) (2,3,5,6,8) (5,7,9) (0,1,2,4,6,8,9) (6) (2,4,6,9) (0,1,2,3,4,7,9) (0,3,4,8) (4,7) (0,1,2,3,4,6,8,9) (0,4) (0,1,2,4,5,8,9) {95,75,106,66,112,74,104,28,92,101}"
m = re.match(r"\[(.*?)\]\s*(.*?)\s*\{(.*?)\}", line)
if not m:
    raise ValueError("Input string does not match expected format")

lights, buttons, joltages = m.groups()
lights = int(lights.replace(".", "0").replace("#", "1")[::-1], 2) # bitmap, invert for the bit indexes to match
buttons  = re.findall(r"\(([^)]*)\)", buttons)
buttons = [ {int(bb) for bb in b.strip().split(",")} for b in buttons]
joltages = [int(j) for j in joltages.split(",")]
row = (lights, buttons, joltages)
row

In [ ]:
from sympy import Matrix, lcm

In [ ]:
if False:
    AA = Matrix([[0,0,0,0,1,1], [0,1,0,0,0,1], [0,0,1,1,1,0], [1,1,0,1,0,0]])
    xx = Matrix([1,3,0,3,1,2])
    bb = Matrix([3,5,4,7])

    
b = Matrix(joltages)

n_rows = max(max(s) for s in buttons) + 1
n_cols = len(buttons)
A = Matrix.zeros(n_rows, n_cols)
for col, rows in enumerate(buttons):
    for row in rows:
        A[row, col] = 1

A_b = A.row_join(b)
A_b

In [ ]:
from sympy import Matrix, solve_linear_system
from sympy import Matrix, symbols
import string

n_unknowns = A_b.shape[-1] - 1

vars_all = symbols(" ".join(string.ascii_lowercase[-n_unknowns:]))
vars_all

In [ ]:
sol = solve_linear_system(A_b, *vars_all)
sol

In [ ]:
solved_vars = set(sol.keys())
free_vars = [vv for vv in vars_all if vv not in solved_vars]
free_vars

In [ ]:

denoms = []
for expr in sol.values():
    denoms.extend(dd.as_numer_denom()[1].as_ordered_factors()
                  for dd in expr.as_ordered_terms())

# simpler robust way:
D = lcm([expr.as_numer_denom()[1] for expr in sol.values()])
D

In [ ]:
if True:
    sol_int = {k: expr.subs({var: D*var for var in free_vars}) for k, expr in sol.items()}
    sol_int
else:
    from sympy import symbols

    # Create fresh integer parameters
    params = symbols(f"k0:{len(free_vars)}", integer=True)  # k0, k1, ...
    subs_free = {fv: D*pv for fv, pv in zip(free_vars, params)}

    sol_int = {k: expr.subs(subs_free) for k, expr in sol.items()}


In [ ]:
from sympy import Matrix

X = Matrix(vars_all)
X_sol = X.subs(sol)
X_sol

In [ ]:
# Prolematic, since also the sum of fractions can be an integer
#X_sol = X_sol.subs({vv: D*vv for vv in free_vars})
#X_sol

In [ ]:
X_particular = X_sol.subs({var: 0 for var in free_vars})
X_particular

In [ ]:
d_vars = {}
for var in free_vars:
    d_var = X_sol.subs({vv: 1 if vv == var else 0 for vv in free_vars}) - X_particular
    d_vars[var] = d_var

#d_vars[z] # last variable

In [ ]:
from operator import add

subs = {free_vars[0]:115, free_vars[1]: 0, free_vars[2]: 11} # x:10,y:0,z: 10, done manually, currently
deltas = [subs[vv]*dvv for (vv, dvv) in d_vars.items()]
X = X_particular + (reduce(add, deltas) if len(deltas) > 0 else  Matrix.zeros(*X_particular.shape))
X

In [ ]:
#X = Matrix([1,6,0,9,9,1,38,7,1,0,17,8,1])

In [ ]:
assert A*X == b
A*X

In [ ]:
sum(X)

In [ ]:
all_vectors = list(d_vars.values())

# TMP

In [ ]:
from sympy import Matrix, Symbol, symbols
from sympy.core.relational import Relational

def build_component_inequalities(
    vx: Matrix,
    vectors: list[Matrix],
    ks: list[Symbol],
) -> list[Relational]:
    """Return scalar inequalities expr[r] >= 0 plus nonnegativity constraints k>=0."""
    if len(vectors) != len(ks):
        raise ValueError("vectors and ks must have the same length")
    if any(v.shape != vx.shape for v in vectors):
        raise ValueError("All vectors must have the same shape as vx")

    expr = vx
    for k, v in zip(ks, vectors, strict=True):
        expr = expr + k * v

    ineqs = [expr[r] >= 0 for r in range(expr.rows)]
    # ineqs += [k >= 0 for k in ks]
    return ineqs

In [ ]:
from sympy import Matrix, symbols

#k0, k1, k2 = symbols("k0 k1 k2", integer=True)

#vx = X_particular
#vectors = [all_vectors[0], all_vectors[1], all_vectors[2]]
#ks = [k0, k1, k2]

ineqs = build_component_inequalities(X_particular, all_vectors, free_vars)
ineqs


In [ ]:
from sympy import And, Max, Min, ceiling, floor, simplify
from sympy.logic.boolalg import BooleanTrue, BooleanFalse


def _iter_constraints(expr_or_list):
    """Yield Relational constraints; skip True; error on False."""
    if isinstance(expr_or_list, tuple) and len(expr_or_list) == 1:
        expr_or_list = expr_or_list[0]

    if isinstance(expr_or_list, And):
        items = expr_or_list.args
    elif isinstance(expr_or_list, list):
        items = expr_or_list
    else:
        items = [expr_or_list]

    for c in items:
        c = simplify(c)
        if c is True or isinstance(c, BooleanTrue):
            continue
        if c is False or isinstance(c, BooleanFalse):
            raise ValueError("Infeasible: a constraint simplified to False.")
        if isinstance(c, Relational):
            yield c
        # else: ignore non-relational boolean structure for this approximation


def approx_bounds_independent_of(
    constraints,
    target: Symbol,
    free_nonneg: list[Symbol],
) -> tuple[list, list]:
    """Return (lower_bounds, upper_bounds) on `target` independent of `free_nonneg`.

    Bounds are conservative and valid for *all* assignments of `free_nonneg >= 0`.
    Assumes constraints are linear/affine in the free variables.
    """
    free_set = set(free_nonneg)
    lbs: list = []
    ubs: list = []

    for ineq in _iter_constraints(constraints):
        # Bring to affine form and isolate target if possible:
        # We handle cases where target appears linearly: a*target + rest >= 0 or <= 0
        lhs = simplify(ineq.lhs - ineq.rhs)
        a = simplify(lhs.coeff(target))
        rest = simplify(lhs - a * target)

        if a == 0:
            continue

        # Convert to target <= rhs or target >= rhs
        # For rel_op:
        #   a*target + rest >= 0  -> target >= (-rest)/a if a>0 else <=
        #   a*target + rest <= 0  -> target <= (-rest)/a if a>0 else >=
        rel = ineq.rel_op

        if rel in (">=", ">"):
            bound_expr = simplify((-rest) / a)
            is_lower = not (a.is_number and a < 0)  # a>0 => lower; a<0 => upper
        elif rel in ("<=", "<"):
            bound_expr = simplify((-rest) / a)
            is_lower = (a.is_number and a < 0)      # a>0 => upper; a<0 => lower
        else:
            continue

        # Now "target >= bound_expr" if is_lower else "target <= bound_expr"
        # Remove dependence on free_nonneg using sign checks
        syms = bound_expr.free_symbols & free_set
        if not syms:
            (lbs if is_lower else ubs).append(bound_expr)
            continue

        ok = True
        subs0: dict[Symbol, int] = {}

        for v in syms:
            coeff = simplify(bound_expr.coeff(v))

            # We only safely handle affine dependence: coeff*v
            # If coeff isn't a plain number, skip for safety.
            if not coeff.is_number:
                ok = False
                break

            if is_lower:
                # target >= a + coeff*v, v>=0.
                # To get a finite k-free lower bound, need coeff >= 0 (min at v=0).
                if coeff < 0:
                    ok = False
                    break
                subs0[v] = 0
            else:
                # target <= a + coeff*v, v>=0.
                # To get a finite k-free upper bound, need coeff <= 0 (max at v=0).
                if coeff > 0:
                    ok = False
                    break
                subs0[v] = 0

        if not ok:
            continue

        bound0 = simplify(bound_expr.subs(subs0))
        if (bound0.free_symbols & free_set):
            continue

        (lbs if is_lower else ubs).append(bound0)

    return lbs, ubs


def approx_integer_range(constraints, target: Symbol, free_nonneg: list[Symbol]):
    """Return (min_int, max_int|None) as a conservative integer range for target."""
    lbs, ubs = approx_bounds_independent_of(constraints, target, free_nonneg)

    lo = Max(*lbs) if lbs else None
    hi = Min(*ubs) if ubs else None

    # If you also know target >= 0, you can enforce it here. Otherwise omit.
    lo_i = ceiling(lo) if lo is not None else None
    hi_i = floor(hi) if hi is not None else None

    return lo_i, hi_i, (lo, hi), (lbs, ubs)


In [ ]:
from collections import deque
queue: deque[str] = deque( [ (i,) for i in range(0, 200)])
final_collection = []
while queue:
    u = queue.popleft()
    if len(u) == len(free_vars):
        final_collection.append(u)
        continue
    
    fixed = {kk: vv for kk, vv in zip(free_vars[:len(u)], u)}
    ineqs_sub = [ineq.subs(fixed) for ineq in ineqs]
    target = free_vars[len(u)]
    free_nonneg = free_vars[len(u)+1:]
    
    _min, _max, *_ = approx_integer_range(ineqs_sub, target, free_nonneg)
    print(_min, _max)
    old_u = list(u)
    for ii in range(_min, _max+1):
        new_u = old_u + [ii]
        queue.append( tuple(new_u) )

In [ ]:
len(final_collection)

In [ ]:
min_size = 10**10
for param_values in final_collection:
    #print(param_values)
    #subs = {free_vars[0]:1, free_vars[1]: 0} # x:10,y:0,z: 10, done manually, currently
    subs = {vv: kk for vv, kk in zip(free_vars, param_values)}
    deltas = [subs[vv]*dvv for (vv, dvv) in d_vars.items()]
    X = X_particular + (reduce(add, deltas) if len(deltas) > 0 else  Matrix.zeros(*X_particular.shape))
    assert A*X == b
    size_x = sum(X)
    if size_x < min_size and all(val >= 0 for val in X) and all(int(val) == val for val in X):
        min_size, X_min = size_x, X
        print(param_values, size_x)

# Backup

In [ ]:
from itertools import product

def nonneg_tuples(n):
    """Generate all n-tuples of nonnegative integers in increasing layers."""
    r = 0
    while True:
        for t in product(range(r + 1), repeat=n):
            yield t
        r += 1


def nonneg_tuples_slow(n, max_n=200):
    """Generate all n-tuples of nonnegative integers in increasing layers."""
    values = [0]*n
    while True:
        yield tuple(values)
        for jj in reversed(range(n)):
            if values[jj] < max_n:
                values[jj] += 1
                break
            values[jj] = 0
        

max_n = 50
gen = nonneg_tuples(len(free_vars), max_n=max_n)

min_size, X_min = 10**100, None
for _ in range(max_n**len(free_vars)):
    param_values = next(gen)
    #print(param_values)
    #subs = {free_vars[0]:1, free_vars[1]: 0} # x:10,y:0,z: 10, done manually, currently
    subs = {vv: kk for vv, kk in zip(free_vars, param_values)}
    deltas = [subs[vv]*dvv for (vv, dvv) in d_vars.items()]
    X = X_particular + (reduce(add, deltas) if len(deltas) > 0 else  Matrix.zeros(*X_particular.shape))
    assert A*X == b
    size_x = sum(X)
    if size_x < min_size and all(val >= 0 for val in X) and all(int(val) == val for val in X):
        min_size, X_min = size_x, X
        print(param_values, size_x)

In [ ]:
from sympy import Matrix, symbols
from sympy.solvers.inequalities import reduce_inequalities

k0, k1, k2 = symbols("k0 k1 k2", integer=True)  # integer constraints (SymPy won't fully enforce them)

vx = X_particular   # example
v0 = all_vectors[0]   # known integer vector
v1 = all_vectors[1]   # known integer vector
v2 = all_vectors[2]   # known integer vector

expr = vx + k0*v0 + k1*v1 + k2*v2

expr = expr.subs({k0: 115})

ineqs = [expr[i] >= 0 for i in range(expr.rows)] #+ [k0>=0, k1>=0]  # componentwise >= 0

ineqs


In [ ]:
region = reduce_inequalities(ineqs, [k1])
region
region.as_set()